In [0]:
%pip install mlflow==2.10.1 langchain==0.1.5 databricks-vectorsearch==0.22 databricks-sdk==0.18.0 mlflow[databricks] pyspark
dbutils.library.restartPython()

  Obtaining dependency information for mlflow==2.10.1 from https://files.pythonhosted.org/packages/e5/10/27177dedbe4ea5ffd81bd90a0149d069f6ba431767f490702198c5021cd5/mlflow-2.10.1-py3-none-any.whl.metadata
  Obtaining dependency information for langchain==0.1.5 from https://files.pythonhosted.org/packages/c1/c3/0e59a0c24e0c61b52271445df55302ab2f3dd8489a365721c7ef7ecaba24/langchain-0.1.5-py3-none-any.whl.metadata
  Obtaining dependency information for databricks-vectorsearch==0.22 from https://files.pythonhosted.org/packages/81/37/3106dc8dedbab9b7560f65088aa7f2894ffa75b5f1928bc472bf42f3b936/databricks_vectorsearch-0.22-py3-none-any.whl.metadata
  Obtaining dependency information for databricks-sdk==0.18.0 from https://files.pythonhosted.org/packages/16/2c/c7920b3806d0c651934a24510f6305958c85df04bacfa65ada3a6cb22e53/databricks_sdk-0.18.0-py3-none-any.whl.metadata
  Obtaining dependency information for mlflow[databricks] from https://files.pythonhosted.org/packages/e5/0f/a1fbf8aa0040244a5

In [0]:
import os

host = "https://" + spark.conf.get("spark.databricks.workspaceUrl")
os.environ['DATABRICKS_TOKEN'] = dbutils.secrets.get(scope="hackathon-databricks", key="hackathon-databricks")

index_name="databricks_hackathon.llm.docs_idx"
host = "https://" + spark.conf.get("spark.databricks.workspaceUrl")

VECTOR_SEARCH_ENDPOINT_NAME="rag_demo"

In [0]:
from databricks.vector_search.client import VectorSearchClient
from langchain_community.vectorstores import DatabricksVectorSearch
from langchain_community.embeddings import DatabricksEmbeddings

embedding_model = DatabricksEmbeddings(endpoint="bge_small_en_v1_5")

def get_retriever(persist_dir: str = None):
    os.environ["DATABRICKS_HOST"] = host
    #Get the vector search index
    vsc = VectorSearchClient(workspace_url=host, personal_access_token=os.environ["DATABRICKS_TOKEN"])
    vs_index = vsc.get_index(
        endpoint_name=VECTOR_SEARCH_ENDPOINT_NAME,
        index_name=index_name
    )

    # Create the retriever
    vectorstore = DatabricksVectorSearch(
        vs_index, text_column="text", embedding=embedding_model
    )
    return vectorstore.as_retriever()



In [0]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_community.chat_models import ChatDatabricks

chat_model = ChatDatabricks(endpoint="dbrx_instruct_v3", max_tokens = 200)

TEMPLATE = """You are an assistant for insurance comparison depending on the physical assets of the properties provided by user. You are answering how to compare the data between the properties and compare between the insurance policies and let us know which will be the best insurance to go with depending on the physical asset information passed by the user queries. If the question is not related to one of these topics, kindly decline to answer. If you don't know the answer, just say that you don't know, don't try to make up an answer. If the question appears to be for an appliance you don't have data on, say so.  Keep the answer as concise as possible.  Provide all answers only in English.
Use the following pieces of context to answer the question at the end:
{context}
Question: {question}
Answer:
"""
prompt = PromptTemplate(template=TEMPLATE, input_variables=["context", "question"])

chain = RetrievalQA.from_chain_type(
    llm=chat_model,
    chain_type="stuff",
    retriever=get_retriever(),
    chain_type_kwargs={"prompt": prompt}
)



[NOTICE] Using a Personal Authentication Token (PAT). Recommended for development only. For improved performance, please use Service Principal based authentication. To disable this message, pass disable_notice=True to VectorSearchClient().


In [0]:
question = {"query": "Insurance issued"}
answer = chain.run(question)
print(answer)

Based on the information provided, here's a comparison of the insurance policies for the two properties in Singapore and the Maldives:

1. Sime Darby Business Center (Singapore):
   - Property type: Office
   - Location: 315 Alexandra Road, Singapore
   - Built-up area: 140,971 sq ft
   - Year built: 1993
   - Insurance policy: Not specified

2. Eclipse Singapore Office (Singapore):
   - Property type: Office
   - Location: 1 Fusionopolis View, Singapore
   - Built-up area: 175,492 sq ft
   - Year built: 2013
   - Insurance policy: Not specified

3. Conrad Maldives Rangali Island (Maldives):
   - Property type: Hotels
   - Location: Conrad Maldives, Alifu Dhaalu Atoll, Rangali Island
   - Built-up area: 1,947,286 sq ft
   -


In [0]:
from mlflow.models import infer_signature
import mlflow
import langchain

mlflow.set_registry_uri("databricks-uc")
model_name = "databricks_hackathon.llm.insurance_model"

with mlflow.start_run(run_name="insurance_comparison_assistant") as run:
    signature = infer_signature(question, answer)
    model_info = mlflow.langchain.log_model(
        chain,
        loader_fn=get_retriever,  # Load the retriever with DATABRICKS_TOKEN env as secret (for authentication).
        artifact_path="chain",
        registered_model_name=model_name,
        pip_requirements=[
            "mlflow==" + mlflow.__version__,
            "langchain==" + langchain.__version__,
            "databricks-vectorsearch",
        ],
        input_example=question,
        signature=signature
    )

2024/10/03 03:13:52 INFO mlflow.models.utils: We convert input dictionaries to pandas DataFrames such that each key represents a column, collectively constituting a single row of data. If you would like to save data as multiple rows, please convert your data to a pandas DataFrame before passing to input_example.
/databricks/python/lib/python3.11/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Successfully registered model 'databricks_hackathon.llm.insurance_model'.
Created version '1' of model 'databricks_hackathon.llm.insurance_model'.
